# Análisis de Data - DIPRO

## Objetivos

1. *Comprender las preguntas:*  
   - Analiza cada pregunta para identificar su intención y significado, ignorando diferencias en la redacción pero centrándote en el contenido o la esencia de la duda.  

2. *Agrupar preguntas similares:*  
   - Encuentra preguntas que tratan sobre lo mismo, incluso si están redactadas de forma diferente.  
   - Crea un campo adicional en la base de datos donde se asigne un identificador único que agrupe todas las preguntas relacionadas con el mismo tema.

3. *Analizar respuestas:*  
   - Identifica respuestas similares o repetidas que aborden la misma cuestión y asigna un identificador único que las agrupe.  

4. *Generar una nueva base de datos:*  
   - Reestructura la base de datos con las siguientes columnas:  
     - Número de registro original.  
     - Pregunta original.  
     - Identificador de grupo de pregunta (esencia de la duda).  
     - Respuesta original.  
     - Identificador de grupo de respuesta (contenido similar).  
   - Incluye en cada registro el número de veces que se repitió una misma duda (grupo de pregunta) y una misma respuesta (grupo de respuesta).  

5. *Resultado final:*  
   - Proporciona un análisis que incluya cuántas dudas distintas existen, cuántas veces se repite cada una y cuántas respuestas similares se generan para estas dudas.  

## Alcance

Se debe asegurar de seguir un enfoque organizado, empleando procesamiento de lenguaje natural (NLP) para identificar patrones semánticos en las preguntas y respuestas. Se debe ignorar las diferencias textuales menores y enfocar esfuerzos en encontrar relaciones semánticas entre los registros. El objetivo es obtener una base de datos clara, depurada y optimizada para el análisis.

## Resultados Esperados

Para realizar un análisis profundo, con diagnóstico y plan de acción, se debe considerar tres (3) puntos:
 
1. Análisis, interpretación y agrupación de los RFIs, por tipo de consulta técnica elaborada (Ejemplo: Consultas sobre Firme de Concreto, Consultas sobre cimentaciones, Consultas sobre estructura del edificio, Consulta sobre Instalación Eléctrica, Consulta sobre terracerías o condiciones de terreno diferentes a mecánica de suelos, Consultas sobre acabados, Consultas sobre refrigeración, etc. etc., etc.)

2. Identificar si las consultas son por omisión del proyecto, por error en el diseño del proyecto, por no haber reflejado la información correcta en el plano correcto, u Otros.

3. Identificar el 80/20 de las consultas que más suceden, así como la causa raíz (por especialidad, por formato, por causa raíz, etc.), y elaborar una propuesta de como atender el problema en corto, mediano y largo plazo. (Hacia Supervisiones, hacia contratistas, hacia Proyectistas, hacia coordinadores de Construcción, hacia Coordinadores de Diseño, etc.)

## Librerías para el Análisis

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import string
import re
import json
import os
import math

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import plotly.graph_objects as go
#style.use('ggplot') or plt.style.use('ggplot')

# Para Clustering
# =============================================================================
from sklearn.cluster import AgglomerativeClustering
# Para cálculos de similitud (en caso de necesitar ajustar)
from sklearn.metrics.pairwise import cosine_similarity

# Importar modelo para embeddings de oraciones (compatible con español)
#=============================================================================
from sentence_transformers import SentenceTransformer

# Preprocesado y modelado
# ============================================================================
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

C:\Users\erivas\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erivas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data

In [2]:
# Open and read the xslx file
xlsx_data = 'D:\Proyectos\DIPRO Agent BOT\Data\RFI All Data v2_limpia.xlsx'
data = pd.read_excel(xlsx_data)
data.head(3)

,ID,Fecha/Hora de creación,Área,Sub Especialidad,Causa Principal,Área impactada,Título del RFI,Pregunta del RFI,Respuesta,Registrado por,RFI Type,Fecha de creación,Mes Creación,Año Creación,Nombre de la tienda,ID.1,Determinante,Banner,Revisión
0,1042790,2023-10-13 09:42:44,Exteriores,Especiales,Requerimiento de Información,Sistemas,ACOMETIDA TELMEX,"Buen dia Respecto a la acometida de telmex, s...",NaN,Abraham Lugo,Design,2023-10-13,10,2023,SAN LUIS POTOSI (RM 2023),1006517.0,2431.0,Walmart Supercenter,0
1,1039329,2023-07-25 16:39:29,Interiores,"Instalaciones Especiales Alarmas, Cctv Y Eas",Requerimiento de Información,Acceso clientes,ALARMADO ACCESO DE CLIENTES,Buen dia Con respecto al alarmado del portico...,NaN,Abraham Lugo,Design,2023-07-25,7,2023,SAN LUIS POTOSI (RM 2023),1006517.0,2431.0,Walmart Supercenter,0
2,1038746,2023-07-11 18:51:51,Interiores,Instalación Eléctrica,Actualización de Prototipo,Otros,ALCANCES LOCALES COMERCIALES,BUEN DIA EN RECORRIDO CON STORE PLANING SE C...,NaN,Abraham Lugo,Design,2023-07-11,7,2023,SAN LUIS POTOSI (RM 2023),1006517.0,2431.0,Walmart Supercenter,0


In [3]:
# Asegurar un identificador de registro (si no existe, usar índice)
data.reset_index(inplace=True)
data.rename(columns={'index': 'Registro'}, inplace=True)

## Preprocesamiento del Texto

Se define una función para limpiar el texto (pasar a minúsculas, quitar puntuación y espacios extra) y se aplica tanto a las preguntas como a las respuestas.

In [4]:
def clean_text(text):
    """Limpia el texto: lo pasa a minúsculas, quita signos de puntuación y espacios extra."""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # elimina puntuación
    text = re.sub(r'\s+', ' ', text).strip()
    return text

### Aplicar la limpieza a las columnas relevantes

In [5]:
data['Pregunta_clean'] = data['Pregunta del RFI'].astype(str).apply(clean_text)
data['Respuesta_clean'] = data['Respuesta'].astype(str).apply(clean_text)

## Cálculo de Embeddings para Preguntas y Respuestas

Utilizamos el modelo de SentenceTransformer (en este ejemplo se usa el modelo multilingüe) para generar representaciones vectoriales que permitan comparar semánticamente las oraciones.

In [6]:
# Se usa un modelo multilingüe optimizado para tareas de similitud
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Generar embeddings para las preguntas y respuestas limpias
question_embeddings = model.encode(data['Pregunta_clean'].tolist())
answer_embeddings = model.encode(data['Respuesta_clean'].tolist())

## Agrupación (Clustering) de preguntas y respuestas

Utilizamos un algoritmo de clustering jerárquico (AgglomerativeClustering) para agrupar oraciones semánticamente similares.
Se define un umbral (por ejemplo, 0.7) que se puede ajustar según la calidad de la agrupación obtenida.

In [7]:
# Agrupación de preguntas
clustering_questions = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.7,       # Este valor se puede ajustar
    metric='cosine',            # Mide la similitud coseno
    linkage='average'
)
data['Pregunta_Group_ID'] = clustering_questions.fit_predict(question_embeddings)

# Agrupación de respuestas
clustering_answers = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.7,       # Se puede ajustar de igual forma
    metric='cosine',
    linkage='average'
)
data['Respuesta_Group_ID'] = clustering_answers.fit_predict(answer_embeddings)


## Cálculo de la frecuencia de ocurrencia en cada grupo

Se cuentan la cantidad de registros que comparten el mismo grupo (tanto para preguntas como para respuestas) y se añade esa información a cada registro.

In [8]:
# Frecuencia de cada grupo de preguntas
group_preg_freq = data.groupby('Pregunta_Group_ID')['Pregunta del RFI'].count().rename('Pregunta_Frequency')

# Frecuencia de cada grupo de respuestas
group_resp_freq = data.groupby('Respuesta_Group_ID')['Respuesta'].count().rename('Respuesta_Frequency')

# Fusionar los recuentos con el DataFrame principal
data = data.merge(group_preg_freq, on='Pregunta_Group_ID', how='left')
data = data.merge(group_resp_freq, on='Respuesta_Group_ID', how='left')

## Reestructuración y exportación de la nueva base de datos

Se crea un DataFrame final que contiene las siguientes columnas:

* Registro: Número de registro original.

* Pregunta del RFI: Pregunta original.

* Pregunta_Group_ID: Identificador del grupo semántico de la pregunta.

* Respuesta: Respuesta original.

* Respuesta_Group_ID: Identificador del grupo semántico de la respuesta.

* Pregunta_Frequency: Número de veces que se repite esa misma duda (grupo).

* Respuesta_Frequency: Número de veces que se repite esa respuesta (grupo).

In [9]:
final_df = data[['Registro', 'Pregunta del RFI', 'Pregunta_Group_ID', 
               'Respuesta', 'Respuesta_Group_ID', 'Pregunta_Frequency', 'Respuesta_Frequency']]

# Exportar el DataFrame final a un archivo Excel
final_df.to_excel('D:\Proyectos\DIPRO Agent BOT\Data\RFI_Data_Analizado.xlsx', index=False)
print("El análisis se ha exportado a 'RFI_Data_Analizado.xlsx'")

El análisis se ha exportado a 'RFI_Data_Analizado.xlsx'


## Análisis exploratorio: 80/20 y diagnóstico inicial

Para identificar el 80/20 de las consultas (las dudas que concentran el 80% del total) se puede calcular la distribución de las frecuencias en los grupos de preguntas. Además, se muestra cuántos grupos (dudas) únicos y grupos de respuestas únicos existen.

In [10]:
# Análisis de número de grupos únicos
n_dudas = final_df['Pregunta_Group_ID'].nunique()
n_respuestas = final_df['Respuesta_Group_ID'].nunique()

print(f"Número total de dudas distintas (grupos de preguntas): {n_dudas}")
print(f"Número total de grupos de respuestas distintas: {n_respuestas}")

# Calcular la frecuencia por grupo de preguntas
freq_preg = final_df.groupby('Pregunta_Group_ID').size().reset_index(name='Count')
freq_preg = freq_preg.sort_values(by='Count', ascending=False)
freq_preg['Cumsum'] = freq_preg['Count'].cumsum()
freq_preg['Cumulative_Percent'] = freq_preg['Cumsum'] / freq_preg['Count'].sum()

print("\nDistribución de las frecuencias de las dudas:")
print(freq_preg)

# Identificar los grupos que representan aproximadamente el 80% de las consultas
top_80 = freq_preg[freq_preg['Cumulative_Percent'] <= 0.8]
print("\nGrupos que representan el 80% de las consultas:")
print(top_80)

Número total de dudas distintas (grupos de preguntas): 27
Número total de grupos de respuestas distintas: 47

Distribución de las frecuencias de las dudas:
    Pregunta_Group_ID  Count  Cumsum  Cumulative_Percent
2                   2  22781   22781            0.981263
11                 11     97   22878            0.985441
20                 20     91   22969            0.989361
5                   5     58   23027            0.991859
10                 10     25   23052            0.992936
8                   8     20   23072            0.993797
4                   4     18   23090            0.994573
7                   7     17   23107            0.995305
1                   1     14   23121            0.995908
9                   9     13   23134            0.996468
22                 22     12   23146            0.996985
26                 26     10   23156            0.997416
24                 24      9   23165            0.997803
13                 13      7   23172          